<a href="https://colab.research.google.com/github/jvataidee/ManualdeAnalisedeDados_Python/blob/master/cap3_estatistica_descritiva_bivariada.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**by: [João Ataíde](https://www.joaoataide.com)**
# **CAP 02: Estatística Descritiva Bivariada**

---



In [1]:
#importar bibliotecas
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import seaborn as sns
import statistics as st
import scipy.stats as stats

## Associações entre duas variáveis qualitativas
Objetivo é avaliar se existe uma relação entre as variáveis qualitativas/categóricas e o grau de assosicação entre elas.

### Tabelas de distribuição conjunta de frequência